In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

25/06/09 22:55:43 WARN Utils: Your hostname, Rams-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
25/06/09 22:55:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 22:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60564)
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/kramkrishnaachary/Library/Python/3.9/l

In [2]:
data = [
    (1, "Aditya"),
    (2, "Arjun"),
    (3, "Rajesh"),
    (4, "Suresh"),
    (5, "Vijay")]

df = spark.createDataFrame(data, ["seat_id", "name"])

df.show(truncate=False)

+-------+------+
|seat_id|name  |
+-------+------+
|1      |Aditya|
|2      |Arjun |
|3      |Rajesh|
|4      |Suresh|
|5      |Vijay |
+-------+------+



In [3]:
from pyspark.sql.functions import *
from pyspark.sql import *

windowSpec = Window.orderBy("seat_id")

df_ranked = df.withColumn("row_number", row_number().over(windowSpec))
df_ranked.show(truncate=False)


25/06/09 22:55:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 22:55:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 22:55:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+------+----------+
|seat_id|name  |row_number|
+-------+------+----------+
|1      |Aditya|1         |
|2      |Arjun |2         |
|3      |Rajesh|3         |
|4      |Suresh|4         |
|5      |Vijay |5         |
+-------+------+----------+



In [9]:
df_swapped = df_ranked.withColumn("swap_seat_id", when(col("row_number") % 2 == 1, lead("seat_id").over(windowSpec))
                                    .when(col("row_number") % 2 == 0, lag("seat_id").over(windowSpec)))

df_swapped.select("seat_id", "name", "swap_seat_id").show(truncate=False)

+-------+------+------------+
|seat_id|name  |swap_seat_id|
+-------+------+------------+
|1      |Aditya|2           |
|2      |Arjun |1           |
|3      |Rajesh|4           |
|4      |Suresh|3           |
|5      |Vijay |NULL        |
+-------+------+------------+



25/06/09 23:06:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 23:06:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 23:06:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 23:06:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/09 23:06:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


25/06/10 00:33:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 919112 ms exceeds timeout 120000 ms
25/06/10 00:33:30 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/10 00:50:54 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o